<a href="https://colab.research.google.com/github/goulartsursula/DisciplinasUERJ/blob/master/Introdu%C3%A7%C3%A3o%20%C3%A0%20Dosimetria/dosimetria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introdução à Dosimetria
Aluna: Ursula Goulart

Professora: Dra. Luciana Coutinho

#Experimento I: Dosimetria com Câmara de Ionização

Objetivos: Determinar a dose absorvida na água a uma profundidade de 10cm, ou seja, profundidade de máxima dose,utilizando a tensão do eletrômetro em -300V.


In [157]:
import numpy as np
import pandas as pd

##Dados

* prof10: Medidade de dosimetria a uma profundidade de 10cm, com tensão de -300V;
* dados_tneg150: Medidade de dosimetria a uma profundidade de 10cm, com tensão de -150V;
* dados_t300: Medidade de dosimetria a uma profundidade de 10cm, com tensão de 300V;



In [158]:
prof10 = np.array([12.64, 12.63, 12.64, 12.63, 12.65])
prof20 = np.array([7.298, 7.294, 7.302, 7.302, 7.287])
dados_t300 = np.array([12.62, 12.63, 12.64, 12.63, 12.64])
dados_tneg150 = np.array([12.63, 12.64, 12.63, 12.62, 12.63])

##Constantes:

* p: pressão medida na sala;
* t: temperatura medida na sala;
* p0: pressão inicial dada pelo roteiro;
* t0: temperatura inicial dada pelo roteiro;
* kelec: fator de correção para a calibração do eletrômetro; 
* ndw: fator de calibração da câmara de ionização;
* hpl: fator de correção para a diferença de fluência de elétrons no plástico comparada com aquela na água na profundidade equivalente;

In [159]:
a0 = 2.337
a1 = -3.636
a2 = 2.299

m1 = -300
m2 = -150
m3 = 300

p = 1.017
p0 = 1.01325
t = 24.5
t0 = 20
ndw = 5.299
kelec = 1
kh = 1
hpl = 1

pdd = 67.7

In [160]:
D10 = np.mean(prof10)
D20 = np.mean(prof20)
tneg300 = np.mean(prof10)
tneg150 = np.mean(dados_tneg150)
t300 = np.mean(dados_t300)

### Cálculo do índice de qualidade do feixe a partir da determinação da razão das doses a 20cm e a 20cm de profundidade (tpr)

In [161]:
rdoses = D20/D10
tpr = (1.2661 * rdoses) - 0.0595

print('rdoses = ', rdoses)
print('tpr = ', tpr)

rdoses =  0.5773540117107137
tpr =  0.6714879142269345


Para determinar o fator de correção que relaciona a qualidade do feixe do usuário e o feixe γ de Co-60 da calibração do laboratório (kqq), é necessário realizar interpolação a partir de ktpr, utilizando a tabela "Beam quality $TPR_{20/10}$

In [162]:
data = [[0.65, 0.994], [0.68, 0.990]]
def interpolation(d, x):
    kqq = d[0][1] + (x - d[0][0]) * ((d[1][1] - d[0][1])/(d[1][0] - d[0][0]))

    return kqq

kqq = interpolation(data, tpr)
print("kqq = ", interpolation(data, tpr))

kqq =  0.9911349447697421


### Fator de correção para a diferença na resposta na câmara de ionização devido à temperatura e pressão em ocondições diferentes das do laboratório (ktp)

In [163]:
ktp = ((273.2 + t)*p)/((273.2 + t0)*p0)
print('ktp = ', ktp)

ktp =  1.0191056495971351


### Fator de correção para a diferença na resposta da câmara de ionização para a falta de completa coleção de cargas devido à recombinação iônica (ks)

In [164]:
ks = a0 + a1*(tneg300/tneg150) + a2*np.power(tneg300/tneg150, 2)
print('ks = ', ks)

ks =  1.0006102652195472


### Fator de correção para a diferença na resposta da câmara de ionização devido ao efeito da mudança de polaridade da tensão aplicada à câmara (kpol)

In [165]:
kpol = (t300 + tneg300)/(2*(tneg300))
print('kpol = ', kpol)

kpol =  0.9997626206678272


### Leitura do dosímetro para a qualidade Q do feixe, corrigida para fatores de influência (mq)

In [166]:
mq = tneg300 * ktp * kelec * kpol * ks * kh
print('mq = ', mq)

mq =  12.884257901661174


### Determinação da dose absorvida na água, na profundidade de referência


In [167]:
dwq_ref = mq * ndw * kqq
print('dwq_ref = ', dwq_ref)

dwq_ref =  67.66843265369516


### Determinação da dose absorvida na água, na profundidade máxima da dose

In [168]:
dwq = dwq_ref/pdd
print('dwq = ', dwq)

dwq =  0.999533717188998
